# Proceso de ETL: Steam_games

### Librerias necesarias

In [1]:
import pandas as pd 
import json
import html
import warnings
warnings.filterwarnings("ignore")

## Extraction y primera exploración: 
Datos de archivo JSON se convierten a Dataframe para examinar su contenido.

In [2]:
 #Leer el archivo JSON
games = []
with open('../DatasetsSTEAM/output_steam_games.json', 'r') as f:
    for line in f.readlines():
        data = json.loads(line)
        games.append(data)

#convertirlo a un dataframe
df_games = pd.DataFrame(games)

In [3]:
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


## Transformación: 
Se observan filas que contienen vacíos en todo el registro. Se eliminan dichos registros.

In [4]:
df_games = df_games.dropna(how='all').reset_index(drop=True)
df_games.shape

(32135, 13)

In [5]:
# Verificamos si existen valores nulos en cada columna
df_games.isnull().sum().sort_values(ascending=False)

publisher       8052
developer       3299
genres          3283
release_date    2067
title           2050
price           1377
specs            670
tags             163
app_name           2
reviews_url        2
id                 2
url                0
early_access       0
dtype: int64

### Verificación de duplicados

In [6]:
# Verificar registros duplicados en la columna 'id'
duplicados = df_games[df_games.duplicated(subset=['id'], keep=False)]
duplicados

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [7]:
# se eliminan los index duplicados 
index_a_eliminar = [14573, 74, 30961]
df_games = df_games.drop(index_a_eliminar)

### Transformación de la columna 'release_date'


Extraemos el año de lanzamiento del juego, para ello se crea una nueva columna con el dato. Luego se elimina la columa 'release_date'.

In [8]:
# Convertir la columna 'release_date' al formato de fecha y hora, manejar errores
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

# Extraer el año y crear una nueva columna 'release_year' de las fechas válidas
df_games['release_year'] = df_games['release_date'].dt.year

# elimina la columna 'release_date'
df_games = df_games.drop('release_date', axis=1)
df_games.head()

,publisher,genres,app_name,title,url,tags,reviews_url,specs,price,early_access,id,developer,release_year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018.0
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018.0
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,2017.0
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017.0
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN


In [9]:
df_games.isnull().sum().sort_values(ascending=False)

publisher       8051
developer       3298
genres          3282
release_year    2351
title           2049
price           1377
specs            669
tags             162
app_name           1
url                0
reviews_url        0
early_access       0
id                 0
dtype: int64

Eliminamos las filas con valores NaN (valores nulos) en la columna 'release_year' y reindexamos el DataFrame

In [10]:
# Eliminar filas con valores NaN en 'release_year' y reindexar el DataFrame
df_games.dropna(subset=['release_year'], inplace=True)
df_games.reset_index(drop=True, inplace=True)

### Transformación columna 'genres'
La columna 'genres' esta formada por una lista de los distintos géneros de los videojuegos. Se necesita crear múltiples filas para cada lista de géneros.

In [11]:
# Divide las filas que contienen listas en la columna 'genres', creando múltiples filas para cada lista de géneros
df_games = df_games.explode('genres')

# Elimina las filas que contienen valores nulos en la columna 'genres'
df_games = df_games.dropna(subset=['genres'])

# Muestra las primeras filas del DataFrame 'df_games' después de realizar las transformaciones anteriores
df_games.head()

,publisher,genres,app_name,title,url,tags,reviews_url,specs,price,early_access,id,developer,release_year
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018.0
0,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018.0
0,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018.0
0,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018.0
0,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018.0


Tambien es necesario que los valores unicos de la columna 'genres' este normalizado

In [12]:
# Obtener valores únicos de la columna 'genres' y ordenarlos de forma ascendente
valores_unicos_genres = sorted(df_games['genres'].unique())
valores_unicos_genres

['Accounting',
 'Action',
 'Adventure',
 'Animation &amp; Modeling',
 'Audio Production',
 'Casual',
 'Design &amp; Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing']

Se observa que la columna 'genres' tiene algunas filas con una representación HTML del carácter '&' en el texto. Vamos a representarlas correctamente como '&'.

In [13]:
# Define una función para corregir los valores en la columna 'genres'
def corregir_amp(text):
    return html.unescape(text) if '&amp;' in text else text

# Aplica la función a la columna 'genres'
df_games['genres'] = df_games['genres'].apply(corregir_amp)

### Columnas 'app_name','specs','early_access','publisher','tags','reviews_url','price', 'developer'
Estas columnas, no se utilizarán para las funciones de la API ni para el modelo de recomendación, por lo tanto se eliminan del dataframe.

In [14]:
# Elimina columnas específicas 
df_games.drop(columns=['app_name','specs','early_access','publisher','tags','reviews_url','price','developer','title','url'],inplace=True)
df_games.head()

,genres,id,release_year
0,Action,761140,2018.0
0,Casual,761140,2018.0
0,Indie,761140,2018.0
0,Simulation,761140,2018.0
0,Strategy,761140,2018.0


Ultimo retoque para un futuro merge con la tabla items que nos permita unificar las tres tablas

In [15]:
# Renombra la columna 'id' a 'item_id' en el DataFrame df_games
df_games.rename(columns={"id":"item_id"},inplace=True)
df_games.reset_index(drop=True, inplace=True)
df_games

,genres,item_id,release_year
0,Action,761140,2018.0
1,Casual,761140,2018.0
2,Indie,761140,2018.0
3,Simulation,761140,2018.0
4,Strategy,761140,2018.0
...,...,...,...
70696,Indie,610660,2018.0
70697,Racing,610660,2018.0
70698,Simulation,610660,2018.0
70699,Casual,658870,2017.0


## Carga del dataset
Se guarda el dataframe transformado como steam_games.parquet

In [16]:
# Guarda el DataFrame df_games en formato Parquet en el archivo 'steam_games.parquet' dentro de la carpeta 'Datasets'
df_games.to_parquet('../Datasets/steam_games.parquet')